In [60]:
import fiona
import numpy as np
import pandas as pd

In [153]:
filename = 'qgis_assets/modulos_automatico_final_new.shp'
shp = fiona.open(filename)
print(shp.schema)

{'properties': {'No.': 'int:10', 'Area (Ha)': 'float:11.2', 'Perimetro(': 'float:11.2', 'Area(m2)': 'float:11.2', 'Perimetr_1': 'float:11.2', 'ID_ARRAY': 'int:18', 'ID_CELL': 'int:18', 'ID_ROW': 'int:18', 'ID_COL': 'int:18'}, 'geometry': 'Polygon'}


In [154]:
# Plot rectangles with matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Change matplotlib style
plt.style.use('default')

font_mapping = {'family': 'Palatino Linotype', 'weight': 'normal', 'size': 11}
plt.rc('font', **font_mapping)


def plot_panels(shp, color_code, draw_text=False, text=None, draw_legend=True, savefig=None):
    fig, ax = plt.subplots()
    max_x = centroid_table_pd['x'].max() + 2
    min_x = centroid_table_pd['x'].min() - 2
    max_y = centroid_table_pd['y'].max() + 2
    min_y = centroid_table_pd['y'].min() - 2
    ax.set_xlim(min_x, max_x)
    ax.set_ylim(min_y, max_y)
    width = 10 * (max_x - min_x) / (max_y - min_y)
    if draw_legend:
        width += 1
    fig.set_size_inches(width, 10)

    # Generate color palette from indices
    palette = np.array(sns.color_palette("hls", len(np.unique(color_code))))

    for (idx, elem) in enumerate(shp):
        x = []
        y = []
        for coord in elem['geometry']['coordinates'][0]:
            x.append(coord[0])
            y.append(coord[1])
        if color_code is None:
            ax.plot(x, y, color='black')
        else:
            ax.plot(x, y, color=palette[color_code[idx]], linewidth=0.5)

    if color_code is not None:
        # Show legend
        for i in range(len(np.unique(color_code))):
            ax.plot([], [], color=palette[i], label=i)
        if draw_legend:
            ax.legend(frameon=False, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

    if draw_text:
        for label in np.unique(color_code):
            random_idx = np.random.choice(np.where(color_code == label)[0])
            centroid = centroid_table[random_idx]
            text_label = label if text is None else text[random_idx]
            ax.text(centroid[0], centroid[1], text_label, fontsize=12, color='white', weight='bold', ha='center',
                    va='center', bbox=dict(facecolor='black', edgecolor='none', alpha=0.8))

    plt.tight_layout()
    if savefig is not None:
        plt.savefig(savefig, dpi=500)
    plt.show()


def plot_centroids(centroids, color_code=None, draw_text=False, plot_legend=True, text_size=12, savefig=None):
    fig, ax = plt.subplots()
    max_x = centroids['x'].max() + 2
    min_x = centroids['x'].min() - 2
    max_y = centroids['y'].max() + 2
    min_y = centroids['y'].min() - 2
    ax.set_xlim(min_x, max_x)
    ax.set_ylim(min_y, max_y)
    fig.set_size_inches(5 * (max_x - min_x) / (max_y - min_y), 5)

    # Generate color palette from indices
    if plot_legend:
        palette = np.array(sns.color_palette("hls", len(np.unique(color_code))))
    else:
        palette = np.array(sns.color_palette("rocket", len(np.unique(color_code))))
    plt.scatter(centroids['x'], centroids['y'], c=color_code)

    if color_code is not None:
        # Show legend
        for i in range(len(np.unique(color_code))):
            ax.plot([], [], color=palette[i], label=i)
        # Place legend outside of plot
        if plot_legend:
            ax.legend(frameon=False, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

    if draw_text:
        for label in np.unique(color_code):
            random_idx = np.random.choice(np.where(color_code == label)[0])
            # From pandas dataframe
            centroid = centroids.iloc[random_idx]
            ax.text(centroid[0], centroid[1], label, fontsize=text_size, color='white', weight='bold', ha='center',
                    va='center', bbox=dict(facecolor='black', edgecolor='none', alpha=0.5))

    plt.tight_layout()
    if savefig is not None:
        plt.savefig(savefig, dpi=500)
    plt.show()

In [155]:
class Panel:
    def __init__(self):
        self.id_array = None
        self.id_panel = None
        self.row = None
        self.col = None
        self.geometry = None
        self.centroid = None

    def compute_centroid(self):
        centroid = np.asarray([0, 0], dtype=np.float64)
        for coord in self.geometry:
            centroid[0] += coord[0]
            centroid[1] += coord[1]
        centroid[0] /= len(self.geometry)
        centroid[1] /= len(self.geometry)
        self.centroid = centroid

    def get_area(self):
        area = 0
        for i in range(len(self.geometry) - 1):
            area += self.geometry[i][0] * self.geometry[i + 1][1] - self.geometry[i + 1][0] * self.geometry[i][1]
        area += self.geometry[-1][0] * self.geometry[0][1] - self.geometry[0][0] * self.geometry[-1][1]
        area = np.abs(area) / 2

        return area

    def get_perimeter(self):
        perimeter = 0
        for i in range(len(self.geometry) - 1):
            perimeter += ((self.geometry[i][0] - self.geometry[i + 1][0]) ** 2 + (self.geometry[i][1] - self.geometry[i + 1][1]) ** 2) ** 0.5
        perimeter += ((self.geometry[-1][0] - self.geometry[0][0]) ** 2 + (self.geometry[-1][1] - self.geometry[0][1]) ** 2) ** 0.5

        return perimeter

In [156]:
def get_array_size(id_array, panel_indices, panel_buffer, norm_x):
    # Calculate bounding box of array
    min_x, max_x, min_y, max_y = 2e32, -2e32, 2e32, -2e32

    for idx in panel_indices:
        for coord in panels[idx].geometry:
            # Rotate geometry according to norm_x
            rotated_geometry = [coord[0] * norm_x[0] + coord[1] * norm_x[1], -coord[0] * norm_x[1] + coord[1] * norm_x[0]]
            if rotated_geometry[0] < min_x:
                min_x = rotated_geometry[0]
            if rotated_geometry[0] > max_x:
                max_x = rotated_geometry[0]
            if rotated_geometry[1] < min_y:
                min_y = rotated_geometry[1]
            if rotated_geometry[1] > max_y:
                max_y = rotated_geometry[1]

    return max_x - min_x, max_y - min_y, [min_x, min_y], [max_x, max_y]


def is_point_in_array(point, norm_x, array_min, array_max):
    rotated_point = [point[0] * norm_x[0] + point[1] * norm_x[1], -point[0] * norm_x[1] + point[1] * norm_x[0]]

    if array_min[0] < rotated_point[0] < array_max[0] and array_min[1] < rotated_point[1] < array_max[1]:
        return True
    else:
        return False

def get_panel_geometry(point, x, width, y, height):
    max_x_max_y = point + x * width / 2 + y * height / 2
    min_x_min_y = point - x * width / 2 - y * height / 2
    max_x_min_y = point + x * width / 2 - y * height / 2
    min_x_max_y = point - x * width / 2 + y * height / 2

    return [max_x_max_y, min_x_max_y, min_x_min_y, max_x_min_y]

In [177]:
panels = []
cluster = dict()

for (idx, elem) in enumerate(shp):
    panel = Panel()
    panel.id_array = elem['properties']['ID_ARRAY']
    panel.id_panel = elem['properties']['ID_CELL']
    panel.row = elem['properties']['ID_ROW']
    panel.col = elem['properties']['ID_COL']
    panel.geometry = elem['geometry']['coordinates'][0]
    panel.compute_centroid()

    # Plot geometry
    # fig, ax = plt.subplots()
    # x = []
    # y = []
    # for (idx, coord) in enumerate(panel.geometry):
    #     x.append(coord[0])
    #     y.append(coord[1])
    #
    #     if idx >= 4:
    #         break
    # ax.plot(x, y, color='black')
    # plt.show()

    panels.append(panel)
    if panel.id_array not in cluster:
        cluster[panel.id_array] = []
    cluster[panel.id_array].append(idx)

In [178]:
threshold_width = 1.1

for key in cluster:
    row_panels = dict()

    for idx in cluster[key]:
        if panels[idx].row not in row_panels:
           row_panels[panels[idx].row] = []
        row_panels[panels[idx].row].append(idx)
        # Sort according to column
        row_panels[panels[idx].row].sort(key=lambda x: panels[x].col)

    # Grid size
    max_cols, max_rows = 0, len(row_panels)
    for row in row_panels:
        if len(row_panels[row]) > max_cols:
            max_cols = len(row_panels[row])

    print(row_panels)

    # Average direction
    average_direction = np.zeros(2)
    num_panels = 0

    for row in row_panels:
        for idx in row_panels[row]:
            vec_x_1 = np.asarray(panels[idx].geometry[1]) - np.asarray(panels[idx].geometry[0])
            vec_x_2 = np.asarray(panels[idx].geometry[2]) - np.asarray(panels[idx].geometry[3])
            # Normalize
            norm_vec_x_1 = vec_x_1 / np.linalg.norm(vec_x_1)
            norm_vec_x_2 = vec_x_2 / np.linalg.norm(vec_x_2)

            average_direction = average_direction + norm_vec_x_1 + norm_vec_x_2
            num_panels += 2

    average_direction = average_direction / num_panels
    norm_x = average_direction / np.linalg.norm(average_direction)
    norm_y = np.asarray([-norm_x[1], norm_x[0]])

    # Size of array
    array_width, array_height, array_min, array_max = get_array_size(key, cluster[key], panels, norm_x)

    # Calculate average width and height of panels
    average_width = 0
    average_height = 0
    num_panels = 0

    for row in row_panels:
        for idx in row_panels[row]:
            average_width += np.linalg.norm(np.asarray(panels[idx].geometry[0]) - np.asarray(panels[idx].geometry[1]))
            average_width += np.linalg.norm(np.asarray(panels[idx].geometry[2]) - np.asarray(panels[idx].geometry[3]))
            average_height += np.linalg.norm(np.asarray(panels[idx].geometry[1]) - np.asarray(panels[idx].geometry[2]))
            average_height += np.linalg.norm(np.asarray(panels[idx].geometry[3]) - np.asarray(panels[idx].geometry[0]))
            num_panels += 2
    average_width /= num_panels
    average_height /= num_panels

    average_distance_x = (array_width - average_width * max_cols) / (max_cols - 1)
    average_distance_y = (array_height - average_height * max_rows) / (max_rows - 1)

    # Minimum point of array
    min_x, min_y = 2e32, 2e32
    for row in row_panels:
        for idx in row_panels[row]:
            if panels[idx].centroid[0] < min_x:
                min_x = panels[idx].centroid[0]
            if panels[idx].centroid[1] < min_y:
                min_y = panels[idx].centroid[1]

    for (row_idx, row) in enumerate(row_panels):
        if len(row_panels[row]) < max_cols:
            # starting_point = np.asarray([min_x + average_width / 2.0, min_y + average_height / 2.0 * row_idx + (average_distance_y * (row_idx - 1) if row_idx > 0 else 0)])
            num_panels = len(row_panels[row])

            # Missing panels - start from the left
            starting_point = panels[row_panels[row][0]].centroid - norm_x * (average_width + average_distance_x / 2.0) - norm_y * average_distance_y / 2.0
            while is_point_in_array(starting_point, norm_x, array_min, array_max):
                # Include new panel
                new_panel = Panel()
                new_panel.id_array = key
                new_panel.row = row
                new_panel.geometry = get_panel_geometry(starting_point, norm_x, average_width, norm_y, average_height)
                new_panel.centroid = starting_point
                panels.append(new_panel)
                num_panels += 1

                starting_point = starting_point - norm_x * (average_width + average_distance_x)

            # Find missing panels in the middle
            for row_i in row_panels:
                for i in range(len(row_panels[row_i]) - 1):
                    # Calculate distance between adjacent panels
                    centroid1 = panels[row_panels[row_i][i]].centroid
                    centroid2 = panels[row_panels[row_i][i + 1]].centroid
                    distance = np.linalg.norm(centroid1 - centroid2)

                    if distance > ((average_width + average_distance_x) * threshold_width):
                        new_centroid = centroid1 + norm_x * (average_width + average_distance_x * 1.5) - norm_y * average_distance_y / 2.0
                        new_panel = Panel()
                        new_panel.id_array = key
                        new_panel.row = row_i
                        new_panel.geometry = get_panel_geometry(new_centroid, norm_x, average_width, norm_y, average_height)
                        new_panel.centroid = new_centroid
                        panels.append(new_panel)
                        num_panels += 1

                        # Insert new panel in the list
                        row_panels[row_i].insert(i + 1, len(panels) - 1)

            # Missing panels - start from the right
            starting_point = panels[row_panels[row][len(row_panels[row]) - 1]].centroid + norm_x * average_distance_x - norm_y * average_distance_y / 2.0
            while num_panels < max_cols:
                starting_point = starting_point + norm_x * (average_width + average_distance_x)
                # Include new panel
                new_panel = Panel()
                new_panel.id_array = key
                new_panel.row = row
                new_panel.geometry = get_panel_geometry(starting_point, norm_x, average_width, norm_y, average_height)
                new_panel.centroid = starting_point
                panels.append(new_panel)
                num_panels += 1
#
#     break
#
# print('Number of missing panels: {}'.format(len(missing_panels)))
# missing_panels_centroids = [elem[0] for elem in missing_panels]
# missing_panels_centroids_pd = pd.DataFrame(missing_panels_centroids, columns=['x', 'y'])
# plot_centroids(missing_panels_centroids_pd, color_code=None, draw_text=False, savefig=None, plot_legend=False, text_size=10)

{3: [681, 686, 793, 803, 782, 765, 746, 603, 584, 566, 551, 565, 675, 654, 637, 617, 629, 143, 124, 134, 109, 87, 100, 206, 183, 196, 173, 156, 8, 20, 0, 1850, 1830, 1843, 78, 55, 65, 46, 24, 33, 395, 375, 388, 368, 348, 357, 466, 445, 426, 418, 269, 252, 231, 243, 223, 332, 312, 324, 304, 281, 294, 1503, 1478], 0: [771, 752, 609, 572, 555, 647, 658, 641, 621, 138, 148, 125, 103, 114, 92, 102, 207, 187, 168, 180, 159, 12, 1862, 3, 1854, 1835, 1844, 79, 60, 70, 47, 25, 38, 399, 379, 360, 372, 350, 457, 469, 449, 429, 409, 421, 276, 255, 235, 218, 228, 336, 316, 295, 308, 285, 1494, 1504, 1480, 1489, 1462, 1471, 1476], 4: [705, 709, 682, 693, 799, 777, 788, 773, 754, 611, 593, 574, 556, 666, 651, 631, 642, 623, 140, 150, 127, 105, 116, 93, 202, 212, 189, 170, 182, 161, 14, 1864, 5, 1859, 1836, 74, 81, 62, 40, 49, 27, 392, 401, 381, 362, 374, 354, 462, 439, 451, 434, 414, 265, 277, 260, 240, 220, 328, 339, 317, 297, 310, 287, 1496], 2: [690, 797, 774, 789, 770, 755, 608, 590, 575, 557, 66

In [179]:
print(panels[1194].id_panel)

1233


In [180]:
# Write back
schema = shp.schema.copy()
input_crs = shp.crs

with fiona.open(filename.split('.')[0] + '_fixed.shp', 'w', 'ESRI Shapefile', schema, input_crs) as output:
    # for elem in shp:
    #     output.write({'properties': elem['properties'], 'geometry': elem['geometry']})

    for panel in panels:
        # Build geometry buffer
        geometry_buffer = [[]]
        for point in panel.geometry:
            geometry_buffer[0].append((point[0], point[1]))

        output.write({'properties': {'No.': -1, 'Area (Ha)': -1, 'Perimetro(': panel.get_perimeter(), 'Area(m2)': panel.get_area(), 'Perimetr_1': panel.get_perimeter(), 'ID_ARRAY': -1, 'ID_CELL': -1, 'ID_ROW': -1, 'ID_COL': -1}, 'geometry': {'type': 'Polygon', 'coordinates': geometry_buffer }})